In [ ]:
import cv2
#Reizing image
def resize(img, height=640):
    img_X_scale = height / img.shape[0]
    new_img_width = int(img.shape[1] * img_X_scale)
    img_resized = cv2.resize(img, (int(new_img_width), int(height)))
    return img_resized

# img = cv2.imread("./FacialImages/person0.jpg")
# cv2.imshow("original image", img)
# cv2.imshow("resized",resize(img))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
#Noise Reduction using Bi-Lateral Filter as edge preservation is critical and runtime is a non factor

# Can create "painting-like" effects if parameters are too strong
# Diameter (d): Size of pixel neighborhood
# sigmaColor: How much differences in color/intensity are tolerated
# sigmaSpace: How much spatial distance is tolerated
# Start with d=9, sigmaColor=75, sigmaSpace=75
# Increase sigmaColor if noise persists
# Decrease sigmaColor if edges become too blurry
# Adjust d based on image resolution (larger for higher-res images)

def BiLateralFitlerDenoise(img, diameter = 9, sigma_color=75, sigma_space=75 ):
    return cv2.bilateralFilter(img, diameter, sigma_color, sigma_space)


# img = cv2.imread("./FacialImages/person0.jpg")
# img = resize(img)

# cv2.imshow("original image", resize(img))
# cv2.imshow("Denoised",BiLateralFitlerDenoise(img))

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [12]:
import numpy as np
def is_grayscale(image):
    if len(image.shape) == 2:
        return True
    if image.shape[2] == 1:#no of colour channels 
        return True
    
    channels = cv2.split(image)
    return np.allclose(channels[0], channels[1]) and np.allclose(channels[0], channels[2]) # checks if the values are close 

In [ ]:
# Enhance Contrast CLAHE

# clipLimit: Controls contrast enhancement
    # Higher values = more contrast but more noise
    # Lower values = less contrast but less noise
    # Typical values: 2.0 - 4.0

# tileGridSize: Size of local regions
    # Smaller tiles = more local enhancement
    # Larger tiles = more global enhancement
    # Typical values: (8,8) or (16,16)
def enhanceContrast(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    if (is_grayscale(img)):#check if gray
        
        enhanced = clahe.apply(img)
        return enhanced
    else:
        # Convert to LAB color space
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        
        # Apply CLAHE to L channel
        enhanced_l = clahe.apply(l)
        
        # Merge and convert back
        enhanced_lab = cv2.merge([enhanced_l, a, b])
        return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)
    
img = cv2.imread("./FacialImages/person0.jpg")
img = resize(img)

cv2.imshow("original image", resize(img))
cv2.imshow("Contrast Enhanced", enhanceContrast(img))

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#minmax normalising 
#TODO Maybe different normalising, min maxing a gray image from 0 to 255 does nothing
def normalise(img):
        new_min = 0
        new_max = 255
        old_min, old_max = img.min(), img.max()
        normalized = (img - old_min) / (old_max - old_min)
        normalized = normalized * (new_max - new_min) + new_min
        return normalized.astype(img.dtype)

img = cv2.imread("./FacialImages/person0.jpg")
img = resize(img)


#Showing hist
normalised = normalise(img)


cv2.imshow("original image", img)
cv2.imshow("Normalised", normalised)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
#Unsharp masking
    # kernel_size (tuple): The size of the Gaussian blur kernel.
    # sigma (float): The standard deviation of the Gaussian blur kernel.
    # amount (float): The amount of sharpening to apply.
    # threshold (int): The minimum brightness change that will be sharpened.

def unsharpMask(img, kernel_size=5, sigma=1.5, amount =1.0, threshold=0):
    blurred = cv2.GaussianBlur(img, (kernel_size, kernel_size), sigma, sigma)

    sharpened = cv2.addWeighted(img, 1 + amount, blurred, -amount, 0)

    # Apply a threshold to the sharpening
    image_sharp = np.where((img - blurred) < -threshold, 0, sharpened)

    return image_sharp

cv2.imshow("original image", resize(img))
cv2.imshow("UnSharpMasked", unsharpMask(resize(img)))

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#vertical alignment
# takes in cropped region of face?
# not sure if this will be necessary
import cv2
def verticalAlign(img):
        # Rotate the image, white mask, and green mask
        grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        edge = cv2.Canny(grey, 200, 255, None, 3)
        
        lines = cv2.HoughLines(edge, 1, np.pi / 180, 200)
        if lines is None:
            print("lines is Non")
            return img
        
        angles = []
        for rho, theta in lines[:, 0]:
            angle = np.degrees(theta) - 90
            if -45 < angle < 45:
                angles.append(angle)

        if len(angles) == 0:
            print("len angles 0")
            return img
        
        rotation = np.mean(angles)
        print("Rotation:", rotation)
        (h, w) = edge.shape[:2]

        center = (w // 2, h // 2)
        rotateMatrix = cv2.getRotationMatrix2D(center, rotation, 1.0)
        
        rotated_img = cv2.warpAffine(img, rotateMatrix, (w, h))
    
    
        return rotated_img

img = cv2.imread("./FacialImages/DJCroppedFace.jpeg")
img = resize(img)
img = enhanceContrast(img)
align = verticalAlign(img)
cv2.imshow("original image", img)
cv2.imshow("Aligned", align)

cv2.waitKey(0)
cv2.destroyAllWindows()

Rotation: -14.182644
